# Long Call Option

### I. Strategy Overview

#### Payoff

![](long_call.jpg)

#### Features:
* Long Delta: option value increases as the underlying price goes up
* Short Theta: option value decreases as time decays
* Long volatility: option value increases as the implied volatility hikes

### II. Strategy Setup

In [1]:
import sys  
sys.path.insert(0, 'C:\\Users\\chena\\Desktop\\Trading\\StrategicOptions\\')
import datetime
# from Functions.OptionBasics import StrategyPerformance
from Functions.mainprg import GetExpiration, GetPayOff, GetBackTesting

####  1. Input symbol of underlying

In [2]:
Ticker = 'MSFT'

#### 2. Specify testing period and find out the expiration dates within

In [3]:
Start_Date = datetime.datetime(2017, 1, 4)
End_Date = datetime.datetime(2018,5, 31)

expiry = GetExpiration(Ticker, Start_Date, End_Date, []).expiration_dates

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### 3. Choose option expiration and  enter dates

In [4]:
entry_date = [datetime.datetime(2017, 1, 4)]
expiration = [datetime.datetime(2018, 5, 18)]

#### 4. Option Type & Strike Price Selection
* Change the strike price to see how it affects the strategy

In [5]:
option_type = ['P', 'P']
action = ['S', 'L']
strike_price = [65, 55]  # Deep in the money
# strike_price = 55  # At the money
# strike_price = 65  # Far out of the money

In [6]:
import numpy as np
np.mean(strike_price)

60.0

### III. Visualization
#### 1. Generate options basic data

In [7]:
payoff = GetPayOff(Ticker, strike_price, entry_date[0], expiration[0], option_type, action)
# payoff.plot_payoff()

AttributeError: 'GetPayOff' object has no attribute 'implied_volatility'


#### 3. Actual Price and Volatility Moves

In [ ]:
exit_date = datetime.datetime(2018, 3, 3)
bt1 = GetBackTesting(Ticker, strike_price[0], expiration[0], entry_date, exit_date, option_type[0], action[0])

#### 4. Returns

In [ ]:
bt1.plot_price_history()

In [ ]:
bt1.plot_option_pnl()

### IV. Trade Journaling

In [ ]:
bt1.strategy_summary

### V. Recurring Strategy Setup
Get in when:
Simplified
* 1st trading day of every month
* Buy next month's expiration

Realistic:
* IV < 0
* Expiration date < 60 & Expiration date > 60
* VIX < 20

In [ ]:
import pandas as pd

bt2 = GetBackTesting(Ticker, strike_price[0], expiration[0], entry_date, exit_date, option_type[0], action[0])
trading_days = bt2.trading_days

trading_days['Year-Month'] = trading_days['Date'].dt.to_period('M') + 1  # +1 to find the expiration date in the next month
dte_seeker = pd.merge(trading_days, expiry, on='Year-Month', how='left')
dte_seeker['DTE'] = (pd.to_datetime(dte_seeker['Trading Day']) - pd.to_datetime(dte_seeker['Date'])).dt.days
dte_seeker.dropna(inplace=True)

In [ ]:
from datetime import timedelta
entry_dates = dte_seeker[dte_seeker['DTE']==43]['Date'].reset_index(drop=True)
expiration_dates = dte_seeker[dte_seeker['DTE']==43]['Trading Day'].reset_index(drop=True)
exit_dates = entry_dates + timedelta(days=25)

# exit_dates = dte_seeker[(dte_seeker['DTE']==21)]['Date']
# exit_dates[exit_dates>entry_dates[0]].reset_index(drop=True)

entry_dates

In [ ]:
import pandas as pd
expiration_dates = pd.to_datetime(expiration_dates)

In [ ]:
strategy_performance = pd.DataFrame([])
i = 0
strike_price = [[55]]
for _ in entry_dates:
    bt = GetBackTesting(Ticker, strike_price[0][0], expiration[0], entry_dates[i], exit_dates[i], option_type[0], action[0])
    strategy_performance = strategy_performance.append(bt.strategy_summary)
    i+=1
    strike_price[0][0] = bt.underlying_price_truncated_['close'][-1]-10

In [ ]:
win_rate = "{:.2%}".format(strategy_performance['Win'].sum()/len(strategy_performance['Win']))
# Total_Cost = "${:,.2f}".format(strategy_performance['Cost Basis'].sum())

In [ ]:
win_rate

In [ ]:
strategy_performance